In [21]:
from osm_integration import OSMManager
from models.Adult import Adult
from datetime import datetime
import folium

In [22]:
# Define the center point and radius for OSMManager
center_point = (48.8588897, 2.3200410217200766)  # Example: Ho Chi Minh City center
radius = 10000  # 10 km radius

# Initialize OSMManager
osm_manager = OSMManager(center_point, radius, network_type="drive")

Loading graph from cache: graph_cache/6c6204fb1da7e7ac4bf3d71ef64620ab.pkl
Scanning for residential...
Scanning for residential near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 473 residential.
Scanning for parks...
Scanning for parks near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 1927 parks.
Scanning for schools...
Scanning for schools near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 3049 schools.
Scanning for workplaces...
Scanning for workplaces near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 7473 workplaces.
Scanning for markets...
Scanning for markets near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 1347 markets.
Scanning for healthcare...
Scanning for healthcare near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 2027 healthcare.
Scanning for play_areas...
Scanning for play_areas near (48.8588897, 2.3200410217200766) within 10000 meters...
Found 2404 play_areas.
Scanning fo

In [23]:
# Create an adult person
adult = Adult(unique_id=1, type="adult", speed=1.4, osm_manager=osm_manager)

Person 1 assigned home at (48.837361699199896, 2.240621278069616).
Person 1 assigned workplace at (48.865446613320145, 2.4371274093158735).
Person 1 assigned gym at (48.8387434, 2.2399337).
Person 1 assigned market at (48.8368245, 2.2385719).
Trajectory built from (48.837361699199896, 2.240621278069616) to (48.865446613320145, 2.4371274093158735) with 234 nodes.
Trajectory built from (48.865446613320145, 2.4371274093158735) to (48.8368245, 2.2385719) with 202 nodes.
Trajectory built from (48.8368245, 2.2385719) to (48.865446613320145, 2.4371274093158735) with 238 nodes.
Trajectory built from (48.865446613320145, 2.4371274093158735) to (48.8387434, 2.2399337) with 204 nodes.
Trajectory built from (48.8387434, 2.2399337) to (48.837361699199896, 2.240621278069616) with 5 nodes.


In [24]:
# Print the assigned waypoints
print("\n--- Assigned Waypoints ---")
for waypoint, coords in adult.waypoints.items():
    if coords:
        print(f"{waypoint.capitalize()}: {coords}")


--- Assigned Waypoints ---
Home: (48.837361699199896, 2.240621278069616)
Workplace: (48.865446613320145, 2.4371274093158735)
Market: (48.8368245, 2.2385719)
Gym: (48.8387434, 2.2399337)


In [25]:
# Print the schedule
print("\n--- Schedule ---")
for movement in adult.schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']} at {movement['start_time']}")


--- Schedule ---
home → workplace at 07:00:00
workplace → market at 12:00:00
market → workplace at 13:00:00
workplace → gym at 18:00:00
gym → home at 20:00:00


In [26]:
# Print the detailed schedule with trajectories
print("\n--- Detailed Schedule ---")
for movement in adult.detail_schedule:
    print(f"{movement['start_waypoint']} → {movement['end_waypoint']}")
    print(f"  Start Time: {movement['start_time']}")
    print(f"  Arrival Time: {movement['arrival_time']}")
    print(f"  Distance: {movement['distance_m']} meters")
    print(f"  Travel Time: {movement['travel_time_s']} seconds")
    print(f"  Route Nodes: {movement['route_nodes']}")


--- Detailed Schedule ---
home → workplace
  Start Time: 07:00:00
  Arrival Time: 10:28:36.888055
  Distance: 17523.643277589726 meters
  Travel Time: 12516.888055421234 seconds
  Route Nodes: [139192418, 4415006638, 4415006635, 4415006625, 4415006632, 243646128, 108270257, 128612880, 108270259, 791974550, 114284127, 432972875, 139264033, 108270264, 139266047, 139262743, 366145557, 260659519, 89163066, 306087402, 306087465, 1027183265, 260660831, 208911944, 208912939, 25077734, 180286853, 25078032, 1755336947, 1757366871, 1755336959, 6956196368, 248107539, 673770416, 673770421, 673770423, 2303848345, 30624766, 673770429, 34909766, 6198530096, 141592723, 116137021, 116137774, 116124379, 116118809, 3865046321, 116118798, 3865046319, 116147457, 110548702, 3633222254, 501556367, 12412514109, 12412514108, 110548697, 360982, 112014705, 139248385, 110548696, 110548695, 139238000, 113494619, 110548693, 138262786, 138263854, 191744889, 110509773, 113494973, 12412435295, 12412435300, 389131719,

In [27]:
# Simulate position at a specific time
current_time = datetime.now().replace(hour=7, minute=30)  # Example time
position = adult.get_position_at_time(current_time)

print("\n--- Simulated Position ---")
print(f"Time: {current_time}")
print(f"Position (latitude, longitude): {position}")


--- Simulated Position ---
Time: 2025-01-12 07:30:45.773997
Position (latitude, longitude): (np.float64(48.83487016006366), np.float64(2.2719091421215136))


In [28]:
# Create a Folium map centered at the person's home
home_coords = adult.waypoints["home"]
folium_map = folium.Map(location=home_coords, zoom_start=13)

# Add waypoints to the map
for waypoint, coords in adult.waypoints.items():
    if coords:
        folium.Marker(
            location=coords, 
            popup=waypoint.capitalize(), 
            icon=folium.Icon(color="blue")
        ).add_to(folium_map)

# Add trajectories to the map
for movement in adult.detail_schedule:
    route_nodes = movement["route_nodes"]
    route_coords = [(osm_manager.nodes.loc[node, "y"], osm_manager.nodes.loc[node, "x"]) for node in route_nodes]
    
    # Draw the route as a polyline
    folium.PolyLine(route_coords, color="red", weight=2.5, opacity=0.8).add_to(folium_map)

# Add current position to the map
folium.Marker(
    location=position,
    popup=f"Current Position at {current_time.time()}",
    icon=folium.Icon(color="green", icon="info-sign"),
).add_to(folium_map)

In [29]:
# Save the map to an HTML file
map_filename = "map/person_movement_map.html"
folium_map.save(map_filename)
print(f"Map saved to {map_filename}")


Map saved to map/person_movement_map.html
